Install and  Import Dependencies

In [1]:
pip install mediapipe opencv-python pandas scikit-learn

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement mediapipe (from versions: none)
ERROR: No matching distribution found for mediapipe


In [4]:
import mediapipe as mp
import cv2
import numpy as np


ModuleNotFoundError: No module named 'mediapipe'

In [ ]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_pose = mp. solutions.pose


In [ ]:
VIDEO_PATH = 'lean.avi'
EXPORT_PATH = 'lean. osv'
MODEL_PATH  =  'lean. pkl'

1XSave Video

In [ ]:
cap = cv2. VideoCapture(3)


height = cap.get (cv2.CAP_PROP_FRAME_HEIGHT)
width =  cap•get(cv2.CAP_PROP_FRAME_WIDTH)
fps = cap.get (cv2.CAP_PROP_FPS)
videoWriter = cv2.VideoWriter (VIDEO_PATH, cv2.VideoWriter_fourcc('P', 'I', 'M', '1'), fps, (int(width), int(height)))
while cap.isOpened():
    ret, frame = cap.read()
    try:
       cv2. imshow('Press', frame)
       videoWriter. write(frame)
    except Exception as e:
        break
    if cv2.waitKey (10) & OxFF== ord('9'):
        break

cap. release()
videoWriter.release()
cv2. destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()

2.Capture Landmarks and export to CSV

In [ ]:
import csv
import os
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
landmarks = ['class']
for val in range(1, 33+1):
    landmarks += ['x({})'.format(val), 'y({})'.format(val), 'z({})'.format(val), 'v({})'.format(val)]


In [ ]:
with open(EXPORT_PATH, mode ='w', newline='') as f:

  csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting =csv. QUOTE_ MINIMAL)
  csv_writer.writerow(landmarks)


In [ ]:
def export_landmark(results, action):
 try:
    keypoints =np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() 
    keypoints.insert(0,action)
    with open(EXPORT_PATH,mode='a',newline='') as f:
         csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting =csv. QUOTE_ MINIMAL)
         csv_writer.writerow(keypoints)
 except Exception as e:
    pass       

In [ ]:
cap = cv2.VideoCapture(FILE_NAME)
    # Initiate holistic model
with mp_pose.Pose(min_detection_confidence =0.5,min_tracking_confidence =0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        # Recolor Feed
        image = cv2.cvtColor (frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        # Make Detections
        results = pose.process(image)
        # Recolor image back to BGR for rendering
        image.flags .writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        mp_drawing. draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius =2)
                                 )
        k =cv2. waitkey(1) 
        if k== ord('a'):
            export_landmark(results, 'wide')
        if k==ord('s'):
            export_landmark(results, 'neutral')
        if k==ord('d'):
            export_landmark(results, 'narrow') 


        cv2. imshow( 'Raw Webcam Feed', image) 
        if cv2.waitKey(1) & 0xFF == ord('q'):
           break
cap.release()
cv2.destroyAllWindows()        


In [ ]:
cap.release()
cv2.destroyAllWindows()

: 

3.Train Customm Model Using Scikit Learn

3.1 Read in Collected Data and Process

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
df=pd.read_csv(EXPORT_PATH)

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
X=df.drop('class',axis=1)#feautures
y=df['class']  #target value

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.3,
random_state =1234)

In [ ]:
y_test

3.2 Train Macjhine Learning Classification Model


In [ ]:
from sklearn.pipeline import make_pipeline 
import  sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression, RidgeClassifier 
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier


In [ ]:
pipelines = {
    'rf': make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb': make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [ ]:
fit_models = {}
for algo, pipeline in pipelines. items():
    model = pipeline. fit(X_train, y_train)
    fit_models[algo]=model


In [ ]:
fit_models

In [ ]:
fit_models['rf'].predict(X_test)

3.3 Evaluate and Serialize model

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score # Accuracy metrics 
import pickle


In [ ]:
for algo, model in fit_models.items():
    yhat = model. predict(X_test)
    print(algo, accuracy_score(y_test.values, yhat),
    precision_score(y_test.values, yhat, average ="weighted"), 
    recall_score(y_test.values, yhat, average="weighted"))

In [ ]:
with open(MODEL_PATH,'wb') as f:
    pickle.dump(fit_models['rf'],f)

4.Make Detections with Model

In [ ]:
with open(MODEL_PATH,'rb') as f:
    model=pickle.load(f)

In [ ]:
cap =cv2. VideoCapture(0)
counter = 0
current_stage = ' '
# Initiate holistic model
with mp_pose. Pose(min_detection_confidence =0.5, min_tracking_confidence =0.5) as pose:
    while cap.isOpened():
            ret, frame = cap.read()
# Recolor Feed
            image = cv2. cvtColor (frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
# Make Detections
            results =pose.process (image)
# Recolor image back to BR for rendering
            image.flags.writeable = True
            image = cv2. cvtColor(image, cv2.COLOR_RGB2BGR)
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius =4), 
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness =2, circle_radius=2)
                                     
                                     )
            try:
                row = np.array([[res.x, res.y, res.z, res.visibility]  for res in results.pose_landmarks.landmark]).flatten()
                X=pd.DataFrame([row],columns=landmarks[1:])
                body_language_class=model.predict(X)[0]
                body_language_prob=model.predict_proba(X)[0]
                print(body_language_class,body_language_prob)
                if (body_language_class =='left' and body_language_prob[body_language_prob.argmax()]>=.7 )| (body_language_class =='right' and body_language_prob[body_language_prob.argmax()]>=.7 )| (body_language_class =='neutral' and body_language_prob[body_language_prob.argmax()]>=.7 ):
                     current_stage=body_language_class 
                elif current_stage != body_language_class  and body_language_prob[body_language_prob.argmax()]>=.7: 
                     current_stage=body_language_class
                     
                     print(current_stage)

                #Get Status box
                cv2.rectangle(image,(0,0),(250,60),(245,117,16),-1)
                #display class
                cv2.putText(image,'CLASS',(95,12),cv2.FONT_HARSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA) 
                cv2.putText(image,body_language_class.split(' ')[0],(90,40),cv2.FONT_HARSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)    
                #Display probabilities
                cv2.putText(image,'PROB',(15,12),cv2.FONT_HARSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
                cv2.putText(image,str(round(body_language_prob[np.argmax(body_language_prob)],2)),(10,40),cv2.FONT_HARSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
            except Exception as e:
                pass
            cv2.imshow("window",image) 
             
            if cv2.waitkey(10) & 0XFF== ord('q'):
                 break
cap.release()
cv2.destroyAllWindows()            
 

                       